# [Getting Started with a Movie Recommendation System](https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system)
* Data Set 1 : [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset)
    * These files contain metadata for all 45,000 movies listed in the Full MovieLens Dataset. The dataset consists of movies released on or before July 2017. Data points include cast, crew, plot keywords, budget, revenue, posters, release dates, languages, production companies, countries, TMDB vote counts and vote averages. This dataset also has files containing 26 million ratings from 270,000 users for all 45,000 movies. Ratings are on a scale of 1-5 and have been obtained from the official GroupLens website.
    * ```kaggle datasets download -d rounakbanik/the-movies-dataset```
* Data Set 2 : [TMDB 5000 Movie Dataset](https://www.kaggle.com/tmdb/tmdb-movie-metadata)
    * ```kaggle datasets download -d tmdb/tmdb-movie-metadata```

## 추천 시스템의 시대
  데이터 수집의 성장으로 새로운 정보화 시대가 열렸다. 데이터를 보다 효율적인 시스템을 만드는데 사용되며 대표적으로 추천 시스템이 있다. 추천 시스템은 inforamtion filtering system의 하나이며 검색 결과의 퀄리티를 향상시키며 검색한 아이템과 더 관련이 있거나 사용자의 검색 기록과 연관된 아이템들을 제공해준다.
  
  추천 시스템은 사용자에게 아이템을 제공했을때 rating 또는 선호도를 예측하는데 사용된다. 대부분의 회사들이 적용하고 있으며 아마존의 경우 고객들에게 상품을 제안하며, youtube는 다음 자동 재생할 video를 페이스북은 좋아할 만 한 페이지, 팔로우할 만 한 사람을 추천한다. 또한, 넷플릭스와 Spotify의 경우 추천엔진의 효율성에 크게 의존하고 있다.
  
![netflix](https://i.kinja-img.com/gawker-media/image/upload/s--e3_2HgIC--/c_scale,f_auto,fl_progressive,q_80,w_800/1259003599478673704.jpg)

해당 커널은 TMDB 5000 Movie Dataset을 이용하여 영화 추천시스템을 만든다. 

## 3가지 추천시스템
* Demographic Filtering (인구 통계학 기반 필터링)
    * 영화의 인기도, 장르 기반으로 모든 사용자에게 일반화 된 추천을 제공함.
    * 유사한 인구학적 통계(나이,성별 등..) 사용자에게 같은 영화가 추천됨.
    * 각 사용자마다 성향이 다르기 때문에 해당 접근법은 너무 단순함.
    * 인기가 많고 호평받는 영화가 일반 시청자가 좋아할 가능성이 더 높음.
* Content Base Filtering (컨텐츠 기반 필터링)
    * 특정 아이템을 기반으로 비슷한 아이템을 추천함.
    * 아이템의 메타데이터(장르, 감독, 요약, 배우 등...)를 사용함
    * 특정 아이템을 본 사람은 비슷한 아이템을 좋아할 가능성이 더 큼.
* Collaborative Filtering (협업 필터링)
   * 비슷한 관심사를 가지는 사람을 찾아 그 사람이 본 다른 컨텐츠를 추천
   * 메타데이터가 필요하지 않음!
   

In [4]:
import pandas as pd
import numpy as np
df1=pd.read_csv('../../data/tmdb-5000-movie-dataset/tmdb_5000_credits.csv')
df2=pd.read_csv('../../data/tmdb-5000-movie-dataset/tmdb_5000_movies.csv')

* df1의 각 feature
    * movie_id : 각 영화가 가지는 고유 키
    * cast : 주연, 조연 이름
    * crew : 감독, 작곡가, 편집, 작가 등의 정보
* df2의 각 feature
    * budget : 예산
    * genre : 액션/코미디/스릴러 등의 장르
    * homepage : 영화 홈페이지 링크
    * id : 영화 고유키 (df1의 movie_id)
    * keywords : 영화와 관련된 키워드 및 태그
    * original_language : 원본 영화 언어
    * original_title : 원본 영화 제목
    * overview : 요약
    * popularity : 영화 인기도
    * production_companies : 제작사
    * production_countries : 제작 국가
    * release_date : 개봉일
    * revenue : 영화 수익
    * runtime : 러닝 타임 (분)
    * status : release or remored
    * tagline : 영화 태그라인
    * title : 영화 제목
    * vote_average : 평균 평점
    * vote_count : 투표수

In [5]:
df1.columns = ['id','title2','cast','crew']
df2 = df2.merge(df1,on='id')

In [6]:
df2.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,title2,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
